In [ ]:
# connect to test.db sqlite, create session, then run "get all users" with session.Query
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from models import Base, User, Location, ExternalAddress, Cast
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

engine = create_engine(os.getenv("PLANETSCALE_URL"))
# if you have the link you can do
# engine = create_engine('mysql+pymysql://...')

In [ ]:
# get all users with bio that contains "coinbase"

with sessionmaker(bind=engine)() as session:
    coinbase_users = session.query(User).filter(
        User.bio_text.contains("coinbase")).all()
    
    for user in coinbase_users:
        print(user.username)